# 개요

# Prepare

# 1. 초기 세팅

## Library & data load

In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [153]:
# 처리용 df와 오리지널 버전 남겨놓음
df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202105.csv",encoding='euc_kr')
org_df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202105.csv",encoding='euc_kr')

# Pre-Processing

## Basic info

In [154]:
df.head()

,품목명,단위,등급,가격,산지,친환경구분,입력일
0,[오이]백다다기,8kg,특(1등),10000,경기 양주시,일반,20210430
1,[오이]백다다기,8kg,특(1등),11700,경기 양주시,일반,20210430
2,[오이]백다다기,18kg,특(1등),28000,경북 상주시,일반,20210430
3,[오이]백다다기,18kg,특(1등),20000,경북 상주시,일반,20210430
4,[오이]백다다기,8kg,특(1등),13000,충남 천안시,일반,20210430


In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454109 entries, 0 to 454108
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   품목명     454109 non-null  object
 1   단위      454109 non-null  object
 2   등급      454109 non-null  object
 3   가격      454109 non-null  int64 
 4   산지      451364 non-null  object
 5   친환경구분   454109 non-null  object
 6   입력일     454109 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 24.3+ MB


In [156]:
# 산지 제외하고는 결측치 없는 것으로 확인
df.isnull().sum()

품목명         0
단위          0
등급          0
가격          0
산지       2745
친환경구분       0
입력일         0
dtype: int64

- 결측치 처리

In [157]:
###
# 양이 많지 않으므로 결측치 제거
df = df.dropna(axis=0)

- 영문명으로 전환

In [158]:
df = df.rename(columns = {'품목명':'prd',
                          '단위':'scale','등급':'class',
                          '가격':'price',
                          '산지':'origin',
                          '친환경구분':'eco',
                          '입력일':'reg_date'})

## column 별 정보

### class

In [159]:
# 이름 모두 영어로 변경
# 9등과 없음 모두 9로 변경
df['class'].value_counts().index

Index(['특(1등)', '상(2등)', '보통(3등)', '9등(등외)', '없음', '5등', '4등', '6등', '7등',
       '8등'],
      dtype='object')

In [160]:
df111 = pd.DataFrame([['특(1등)', 1], ['상(2등)',2], ['보통(3등)',3], ['9등(등외)',9], ['없음',9], ['5등',5], ['4등',4], ['6등',6], ['7등',7],['8등',8]],columns=['class','new_class'])

In [161]:
df2 = df.merge(df111)
df2 = df2.drop(columns='class')

In [162]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[오이]백다다기,8kg,10000,경기 양주시,일반,20210430,1
1,[오이]백다다기,8kg,11700,경기 양주시,일반,20210430,1
2,[오이]백다다기,18kg,28000,경북 상주시,일반,20210430,1
3,[오이]백다다기,18kg,20000,경북 상주시,일반,20210430,1
4,[오이]백다다기,8kg,13000,충남 천안시,일반,20210430,1
...,...,...,...,...,...,...,...
451359,[양파]양파(일반),15kg,7700,전남 무안군,일반,20210529,8
451360,[토마토]대저토마토,2.5kg,4000,경남 김해시,일반,20210529,8
451361,[토마토]대저토마토,2.5kg,5000,경남 김해시,일반,20210529,8
451362,[기타근채류]기타근채류(기타),15kg,7300,경북,일반,20210531,8


### eco

In [163]:
# 영문명으로 전환
# 전환기 -> 유기농 재배로 전환중
df2.loc[df2["eco"]=="일반","eco"] = "normal"
df2.loc[df2["eco"]=="우수농산물","eco"] = "good_prd"
df2.loc[df2["eco"]=="무농약","eco"] = "ecofriendly"
df2.loc[df2["eco"]=="유기농","eco"] = "organic"
df2.loc[df2["eco"]=="품질인증","eco"] = "certified"
df2.loc[df2["eco"]=="전환기","eco"] = "ing"

In [164]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[오이]백다다기,8kg,10000,경기 양주시,normal,20210430,1
1,[오이]백다다기,8kg,11700,경기 양주시,normal,20210430,1
2,[오이]백다다기,18kg,28000,경북 상주시,normal,20210430,1
3,[오이]백다다기,18kg,20000,경북 상주시,normal,20210430,1
4,[오이]백다다기,8kg,13000,충남 천안시,normal,20210430,1
...,...,...,...,...,...,...,...
451359,[양파]양파(일반),15kg,7700,전남 무안군,normal,20210529,8
451360,[토마토]대저토마토,2.5kg,4000,경남 김해시,normal,20210529,8
451361,[토마토]대저토마토,2.5kg,5000,경남 김해시,normal,20210529,8
451362,[기타근채류]기타근채류(기타),15kg,7300,경북,normal,20210531,8


### reg_date

In [165]:
df2['reg_date'] = df2['reg_date'].astype('str')

In [166]:
df2['reg_date'] = pd.to_datetime(df2['reg_date'].astype('str'),format='%Y%m%d')

In [167]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[오이]백다다기,8kg,10000,경기 양주시,normal,2021-04-30,1
1,[오이]백다다기,8kg,11700,경기 양주시,normal,2021-04-30,1
2,[오이]백다다기,18kg,28000,경북 상주시,normal,2021-04-30,1
3,[오이]백다다기,18kg,20000,경북 상주시,normal,2021-04-30,1
4,[오이]백다다기,8kg,13000,충남 천안시,normal,2021-04-30,1
...,...,...,...,...,...,...,...
451359,[양파]양파(일반),15kg,7700,전남 무안군,normal,2021-05-29,8
451360,[토마토]대저토마토,2.5kg,4000,경남 김해시,normal,2021-05-29,8
451361,[토마토]대저토마토,2.5kg,5000,경남 김해시,normal,2021-05-29,8
451362,[기타근채류]기타근채류(기타),15kg,7300,경북,normal,2021-05-31,8


### price

In [168]:
# 계산을 위해 scale 값 미리 변경
# 현재 값들은 string이므로 float 형태로 변환할 필요가 있음
# 몇몇 이상치 처리도 진행
def scale_checker(scale):
  if scale.startswith('.')==True:
    return format(float('0'+scale[:-2]),'.2f')
  else:
    return format(float((scale[:-2])),'.2f')
df2['scale'] = df2['scale'].apply(scale_checker)
df2['scale'] = df2['scale'].astype('float')
# df2['scale'] = format(round(df2['scale'].apply(pd.to_numeric),2),'.2f')

In [169]:
df2['scale'].unique()

array([  8.  ,  18.  ,  10.  ,  20.  ,  21.  ,   1.  ,   0.6 ,   4.  ,
        15.  ,   2.  ,   5.  ,   1.5 ,  16.  ,  12.  ,  13.  ,   6.  ,
         9.6 ,   0.2 ,   3.  ,   3.5 ,   2.5 ,   0.7 ,   0.5 ,   0.3 ,
         1.8 ,   1.2 ,  14.  ,  11.  ,   7.  ,   9.  ,   0.8 ,  11.5 ,
         8.2 ,   7.3 ,   8.5 ,  17.  ,  19.  ,   3.6 ,   8.01,  23.  ,
         4.5 ,   7.5 ,   9.07,   0.25,   0.01,   0.05,   0.13,   0.35,
        22.  ,   4.7 ,   2.4 ,   2.8 ,   7.7 ,   5.8 ,  11.3 ,   4.8 ,
         5.3 ,   3.2 ,   1.6 ,   1.1 ,   5.5 ,  13.5 ,   5.2 ,   0.12,
         0.4 ,   0.1 ,   5.01,  10.4 ,  22.68,   2.9 ,   6.5 ,   0.15,
        24.  ,   2.7 ,   2.1 ,   4.2 ,  12.2 ,   1.01,  14.2 , 133.  ,
         1.9 ,  28.  ,  15.01, 999.  ,   3.7 ,   3.8 ,   3.3 ,   4.4 ,
         1.3 ,  68.5 ,  40.  ,  53.5 ,  32.5 ,  77.  ,  80.5 ,  72.  ,
        10.8 ,   2.6 ,   1.4 ,  25.  ,  20.01,   9.7 ,   4.9 ,   6.2 ,
         8.3 ,   6.9 ,  88.7 ,  90.4 ,  85.4 ,  89.6 ,   5.6 ,   6.4 ,
      

In [170]:
# 1원짜리 거래 발견, 너무 거래값이 작은 값들은 삭제
pd.options.display.float_format = '{:.5f}'.format
df2['price'].describe()

count    451364.00000
mean      19649.46123
std       35343.29549
min           0.00000
25%        7050.00000
50%       13000.00000
75%       24000.00000
max     9293000.00000
Name: price, dtype: float64

In [171]:
df2 = df2.drop(df2[df2['price']<500].index)

In [172]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[오이]백다다기,8.00000,10000,경기 양주시,normal,2021-04-30,1
1,[오이]백다다기,8.00000,11700,경기 양주시,normal,2021-04-30,1
2,[오이]백다다기,18.00000,28000,경북 상주시,normal,2021-04-30,1
3,[오이]백다다기,18.00000,20000,경북 상주시,normal,2021-04-30,1
4,[오이]백다다기,8.00000,13000,충남 천안시,normal,2021-04-30,1
...,...,...,...,...,...,...,...
451359,[양파]양파(일반),15.00000,7700,전남 무안군,normal,2021-05-29,8
451360,[토마토]대저토마토,2.50000,4000,경남 김해시,normal,2021-05-29,8
451361,[토마토]대저토마토,2.50000,5000,경남 김해시,normal,2021-05-29,8
451362,[기타근채류]기타근채류(기타),15.00000,7300,경북,normal,2021-05-31,8


In [173]:
# kg 당 가격 생성

df2['price_kg'] = round((df2['price']/df2['scale']),2)

### prd
- 품목명 정리
- 품목 종류 정리

In [174]:
# 대부분 [상품 종류]상품명 으로 정리되어 있는걸 알 수 있음
df2['prd'].value_counts()

[오이]백다다기              15896
[호박]애호박                7374
[참외]참외(기타)             6984
[딸기]설향                 6772
[블루베리]블루베리             6269
                      ...  
[풋고추]풋고추(기타)              1
[무]무말랭이                   1
[무]무(순무)                  1
[아스파라가스]아스파라거스(일반)        1
[기타채소]깻순(바라)              1
Name: prd, Length: 1345, dtype: int64

In [175]:
# 정리를 위해 상품명 삭제
def prd_sort(words):
  p = re.compile('[ㄱ-힣]+')
  word = re.search(p,words)
  return word.group()

df2['prd'] = df2['prd'].apply(prd_sort)

In [176]:
# 기타 품목 삭제
df2 = df2.drop(df2[df2['prd']=='기타'].index)

In [177]:
# 국산,수입산,깐,기타 삭제
def prd_sort2(words):
  p = re.compile('[^국산수입깐기타]+')
  word = re.search(p,str(words))
  return word.group()

df2["prd"] = df2["prd"].apply(prd_sort2)


In [178]:
# 몇몇 품목들 정리가 필요
a = sorted(df2['prd'].unique())
for i in range(len(a)):
  if i%15 != 0:
    print(a[i],end=' ')
  else:
    print(a[i],end=' ')
    print()

가리비 
가물치 가오리 가오리채 가자미 가죽나물 가지 간재미 갈치 감 감귤 감자 갑오징어 갓 강낭콩 개조개 
갯가재 갯장어 건고구마순 건고추 게 게지 겨자 고 고구마 고구마순 고둥 고들빼 고등어 고사리 고추 
고추잎 곤달비 곤드레나물 골뱅이 과실류 과일 과일류 과채 굴 귤 그레이프푸룻 근대 근채류 금감 깻잎 
꼴뚜 꽁치 꽈리고추 낙지 냉이 논우렁이 눈볼대 느 다래 다래순 단감 달래 닭새우 당귀 당근 
대구 대구포 대추 대파 더덕 도라지 돈나물 돌나물 돗나물 동자개 동죽 두류 두릅 두리안 듀리안 
딸 땅콩 람부탄 레몬 리치 마늘 만가닥 만감 맛 망고 망고스턴 망고스틴 매생이 매실 머위대 
메 메론 메밀 멸치 명태 명태곤니 명태알 명태포 모시조개 무 무순 무우 문어 문어바지락 미꾸라지 
미나리 미더덕 민들레 민물새우 바나나 바지락 박 방울토마토 방풍 방풍나물 배 배추 백새우 백합 뱀장어 
버섯 버섯류 버찌 벗 병어 보리 복숭아 봉지굴 부세 부지갱이 부추 붉은고추 붕어 붕장어 브로커리 
브로코리 블루베리 비름 비트 빈스 빨간양배추 삐틀이 사과 살구 삼 삼나물 삼치 상어 상추 새꼬막 
새송이 새싹 새우 새우살 새조개 샐러리 생강 생고추 생채 서대 석류 선동오징어 세러리 셀러리 소라 
속새 숙주나물 시금치 신선초 실파 쌈채류 쑥 쑥갓 씀바귀 아귀 아보카도 아보카드 아스파라가스 아스파라거스 아욱 
알 알로애 알배 앵두 야자 양배추 양상추 양송이 양채 양채류 양파 어류 얼갈이 얼갈이배추 엄나무 
연근 연어 열무 염고등어 엽경채류 엽채 엽채류 오가피 오디 오렌지 오만둥이 오이 오징어 오징어몸통 오징어원양 
오징어채 오징어채원양 오크라 옥 완두콩 용과 우럭조개 우렁 우렁쉥이 우렁이 유채 임연 잉어 자두 자몽 
잔대 재첩 적겨자 적근대 적새우 적채 전복 절단 절단낙지 죽순 중하 쪽파 쭈꾸미 참나물 참다래 
참당귀 참돔 참외 참조 참죽나무순 채소 청경채 청어 체리 총각무 취나물 치커리 칠게 칼라후라워 칼리플라워 
칼리후라워 케일 코라비 코코넛 콜라비 콩 콩나물 키위 키조개 키조개

In [179]:
### 예시
# df2[(df2['prd']=='유채') & (df2['price']==4500)]['prd'].unique()
# df2[df2['prd']=='유채']['prd'].unique()

In [180]:
### 원래 단어 뭐였는지 검색
# org_df[org_df['품목명'].str.contains('느')]['품목명'].unique()

In [181]:
### 현재 이름 검색
# df2[df2['prd']=='느']

In [190]:
###
# JG

# 고 -> 고수로 변경
df2.loc[df2["prd"]=="고","prd"] = "고수"
# 과실류, 과일류, 과채, 채소 -> 과채류로 통합
df2.loc[df2["prd"]=="과실류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일야채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일과채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일","prd"] = "과채류"
df2.loc[df2["prd"]=="과채","prd"] = "과채류"
df2.loc[df2["prd"]=="채소","prd"] = "과채류"
# 고들빼 -> 고들빼기로 변경
df2.loc[df2["prd"]=="고들빼","prd"] = "고들빼기"
# 꼴뚜 -> 꼴두기로 변경
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
# 그레이프푸룻 -> 자몽으로 통합
df2.loc[df2["prd"]=="그레이프푸룻","prd"] = "자몽"

# 느 -> 느타리버섯으로 통합
df2.loc[df2["prd"]=="느","prd"] = "느타리버섯"

# 대구 관련 품목들 정리
df2.loc[df2["prd"]=="대구고니","prd"] = "대구"
df2.loc[df2["prd"]=="대구머리","prd"] = "대구"
df2.loc[df2["prd"]=="대구포","prd"] = "대구"
# 듀리안 -> 두리안으로 통합
df2.loc[df2["prd"]=="듀리안","prd"] = "두리안"
# 딸 -> 딸기로 변경
df2.loc[df2["prd"]=="딸","prd"] = "딸기"

# 만가닥 -> 만가닥버섯으로 변경
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
# 맛 -> 맛조개로 변경
df2.loc[df2["prd"]=="맛","prd"] = "맛조개"
# 망고스턴 -> 망고스틴으로 수정
df2.loc[df2["prd"]=="망고스턴","prd"] = "망고스틴"
# 메 -> 메기로 변경
df2.loc[df2["prd"]=="메","prd"] = "메기"
# 명태 관련 품목들 정리
df2.loc[df2["prd"]=="명태곤니","prd"] = "명태"
df2.loc[df2["prd"]=="명태알","prd"] = "명태"
df2.loc[df2["prd"]=="명태포","prd"] = "명태"
# 무우 -> 무로 통합
df2.loc[df2["prd"]=="무우","prd"] = "무"

# 방풍 -> 방풍나물로 통합
df2.loc[df2["prd"]=="방풍","prd"] = "방풍나물"
# 버섯류 -> 버섯으로 변경
df2.loc[df2["prd"]=="버섯","prd"] = "버섯"
# 벗,버찌 -> 체리로 통합
df2.loc[df2["prd"]=="벗","prd"] = "체리"
df2.loc[df2["prd"]=="버찌","prd"] = "체리"
# 브로커리, 브로코리 -> 브로콜리로 통합
df2.loc[df2["prd"]=="브로커리","prd"] = "브로콜리"
df2.loc[df2["prd"]=="브로코리","prd"] = "브로콜리"

# 삼 -> 수삼으로 변경
df2.loc[df2["prd"]=="삼","prd"] = "수삼"
# 새우살 -> 새우로 변경
df2.loc[df2["prd"]=="새우살","prd"] = "새우"
# 세러리, 셀러리 -> 샐러리로 통합
df2.loc[df2["prd"]=="세러리","prd"] = "샐러리"
df2.loc[df2["prd"]=="셀러리","prd"] = "샐러리"
# 생고추 -> 고추로 통합
df2.loc[df2["prd"]=="생고추","prd"] = "고추"

# 아보카드 -> 아보카도로 변경
df2.loc[df2["prd"]=="아보카드","prd"] = "아보카도"
# 아스파라가스 -> 아스파라거스로 변경
df2.loc[df2["prd"]=="아스파라가스","prd"] = "아스파라거스"
# 알 -> 알타리무로 변경
df2.loc[df2["prd"]=="알","prd"] = "알타리무"
# 알배 -> 알배기로 변경
df2.loc[df2["prd"]=="알배","prd"] = "알배기"
# 양채 -> 양채류로 통합
df2.loc[df2["prd"]=="양채","prd"] = "양채류"
# 얼갈이 -> 얼갈이배추로 통합
df2.loc[df2["prd"]=="얼갈이","prd"] = "얼갈이배추"
# 엽채류, 엽채 -> 엽경채류로 통합
df2.loc[df2["prd"]=="엽채류","prd"] = "엽경채류"
df2.loc[df2["prd"]=="엽채","prd"] = "엽경채류"
# 오징어 관련 품목들 정리
df2.loc[df2["prd"]=="오징어다리","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어원양","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채","prd"] = "오징어"
df2.loc[df2["prd"]=="활복오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채원양","prd"] = "오징어"

# 옥, 풋옥 -> 옥수수로 정리
df2.loc[df2["prd"]=="옥","prd"] = "옥수수"
df2.loc[df2["prd"]=="풋옥","prd"] = "옥수수"
# 임연 -> 임연수어로 정리
df2.loc[df2["prd"]=="임연","prd"] = "임연수어"


# 절단 -> 절단쭈꾸미 -> 쭈꾸미로 정리
df2.loc[df2["prd"]=="절단","prd"] = "쭈꾸미"
# 조개살 -> 조개로 정리
df2.loc[df2["prd"]=="조개살","prd"] = "조개"

# 참조 -> 참조기로 정리
df2.loc[df2["prd"]=="참조","prd"] = "참조기"

# 칼라후라워,칼리플라워, 칼리후라워 -> 컬리플라워로 통합
df2.loc[df2["prd"]=="칼라후라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리플라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리후라워","prd"] = "컬리플라워"
# 코라비 -> 콜라비로 통합
df2.loc[df2["prd"]=="코라비","prd"] = "콜라비"
# 키조개날개 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개날개","prd"] = "키조개"

# 팽이 -> 팽이버섯으로 통합
df2.loc[df2["prd"]=="팽이","prd"] = "팽이버섯"
# 파세리 -> 파슬리로 변경
df2.loc[df2["prd"]=="파세리","prd"] = "파슬리"
# 표고 -> 표고버섯으로 통합
df2.loc[df2["prd"]=="표고","prd"] = "표고버섯"
# 포장바지락 -> 바지락으로 변경
df2.loc[df2["prd"]=="포장바지락","prd"] = "바지락"

### 일단 여기까지 실행
# JY
#각굴 -> 굴로 통합
df2.loc[df2["prd"]=="각굴","prd"] = "굴"
#가오리채 -> 가오리로 통합
df2.loc[df2["prd"]=="가오리채","prd"] = "가오리"
# 게지 -> 키조개로 통합
df2.loc[df2["prd"]=="게지","prd"] = "키조개"
#고 -> 고구마로 통합
df2.loc[df2["prd"]=="고","prd"] = "고구마"
#꼴뚜 -> 꼴뚜기로 통합
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
#돗나물 -> 돌나물로 통합
df2.loc[df2["prd"]=="돗나물","prd"] = "돌나물"
#레드쉬 -> 레디쉬로 통합
df2.loc[df2["prd"]=="레드쉬","prd"] = "레디쉬"
#만가닥 -> 만가닥버섯으로 통합
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
#메 -> 메기로 통합
df2.loc[df2["prd"]=="메","prd"] = "메기"
#봄동 -> 봄동배추로 통합
df2.loc[df2["prd"]=="봄동","prd"] = "봄동배추"
#봉지굴 -> 굴로 통합
df2.loc[df2["prd"]=="봉지굴","prd"] = "굴"
#박 -> 수박으로 통합
df2.loc[df2["prd"]=="박","prd"] = "수박"
#백조 -> 조기로 통합 
df2.loc[df2["prd"]=="백조","prd"] = "조기"
#벅굴 -> 벚굴로 통합
df2.loc[df2["prd"]=="벅굴","prd"] = "벚굴"
#비 -> 비타민으로 통합
df2.loc[df2["prd"]=="비","prd"] = "비타민"
#빈스 -> 콩으로 통합
df2.loc[df2["prd"]=="빈스","prd"] = "콩"
#삐틀이 -> 고둥으로 통합
df2.loc[df2["prd"]=="삐틀이","prd"] = "고둥"
#새송이 -> 새송이버섯으로 통합
df2.loc[df2["prd"]=="새송이","prd"] = "새송이버섯"
#세트 -> 과채류로 통합
df2.loc[df2["prd"]=="세트","prd"] = "과채류"
#알 -> 알로애로 통합
df2.loc[df2["prd"]=="알","prd"] = "알로에"
#엄나무 -> 음나무로 통합
df2.loc[df2["prd"]=="엄나무","prd"] = "음나무"
#염고등어 -> 고등어로 통합
df2.loc[df2["prd"]=="염고등어","prd"] = "고등어"
#오만둥이 -> 미더덕으로 통합
df2.loc[df2["prd"]=="오만둥이","prd"] = "미더덕"
#오징어 관련 품목들 정리
df2.loc[df2["prd"]=="선동오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어몸통","prd"] = "오징어"
#우렁쉥이 -> 멍게로 통합
df2.loc[df2["prd"]=="우렁쉥이","prd"] = "멍게"
#우뭇가시리 -> 우뭇가사리로 통합
df2.loc[df2["prd"]=="우뭇가시리","prd"] = "우뭇가사리"
#적채 -> 적양배추로 통합
df2.loc[df2["prd"]=="적채","prd"] = "적양배추"
#절단낙지 -> 낙지로 통합
df2.loc[df2["prd"]=="절단낙지","prd"] = "낙지"
#조 -> 조개로 통합
df2.loc[df2["prd"]=="조","prd"] = "조개"
#청어원양 -> 청어로 통합
df2.loc[df2["prd"]=="청어원양","prd"] = "청어"
#키조개꼭지 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개꼭지","prd"] = "키조개"

# 암꽃게, 숫꽃게 -> 꽃게로 통합
df2.loc[df2["prd"]=="암꽃게","prd"] = "꽃게"
df2.loc[df2["prd"]=="숫꽃게","prd"] = "꽃게"
# 우렁 -> 우렁이로 통합
df2.loc[df2["prd"]=="우렁","prd"] = "우렁이"
# 알로애 -> 알로에로 변경
df2.loc[df2["prd"]=="알로애","prd"] = "알로에"
# 피마자잎 -> 피마자로 변경
df2.loc[df2["prd"]=="피미자잎","prd"] = "피마자"
# 조미제품 삭제
df2.drop(df2[df2['prd']=='조미제품'].index,inplace=True)

In [191]:
# 기존 products 리스트와 비교
ex_prd = pd.read_csv("../products.csv")
df2[~df2['prd'].isin(ex_prd['prd'])]['prd'].unique()

array(['매실', '향신료', '옥수수', '건고구마순', '향나물', '오크라', '함초', '갯장어', '멸치', '상어',
       '학꽁치', '황강달이', '서대'], dtype=object)

In [192]:
# 물품 리스트 최신화
new_prd = pd.DataFrame(df2['prd'].unique(),columns=['prd'])
# products = ex_prd.append(new_prd[~new_prd['prd'].isin(ex_prd['prd'])],ignore_index=True)
products = ex_prd.append(new_prd,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_8897/2619460125.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products = ex_prd.append(new_prd,ignore_index=True)


In [193]:
products.drop(columns=['Unnamed: 0'],inplace=True)
products.drop_duplicates(inplace=True)
products.reset_index(inplace=True,drop=True)

In [194]:
# 다음 작업을 위해 저장
products.to_csv('../products.csv')

In [195]:
# 전체 거래량과 품목량
# 비교를 위해 분포 파악
# 거래량이 너무 적은 품목은 삭제할 필요성이 있음

prd_lst = (df2['prd'].value_counts()).to_frame()
prd_lst.describe()

,prd
count,247.00000
mean,1823.82591
std,4771.89276
min,1.00000
25%,11.50000
50%,97.00000
75%,1021.50000
max,34365.00000


In [196]:
# 50개 미만 품목은 비교가 힘드므로 삭제
# 줄어든 종류만 남기기

prd_lst = prd_lst[prd_lst['prd']>=50]
prd_lst.reset_index(inplace=True)
prd_lst = prd_lst.rename(columns = {'index':'prd','prd':'count'})

In [197]:
df3 = df2.merge(prd_lst)
df3[df3['count']>=50]['prd'].value_counts()

버섯     34365
오이     27714
토마토    26159
고추     22702
딸기     22564
       ...  
알배기       56
고추잎       55
용과        50
자몽        50
겨자        50
Name: prd, Length: 148, dtype: int64

In [198]:
# 줄어든 것 확인했으므로 count column은 삭제
df3 = df3.drop(columns='count')

### scale

In [199]:
np.set_printoptions(precision=2, suppress=True)
df3['scale'].unique()

array([  8.  ,  18.  ,  10.  ,  20.  ,  21.  ,   1.  ,   0.6 ,   4.  ,
        15.  ,  16.  ,  15.01,  20.01,  55.  ,  24.  ,  40.  ,  10.01,
        27.  ,  32.  ,  13.  ,   5.  ,   2.  ,   7.  ,   6.  ,   3.5 ,
         3.  ,   4.5 ,   1.5 ,   0.3 ,   2.5 ,   5.5 ,   0.5 ,  14.  ,
        17.  ,  12.  ,  11.  ,   9.  ,   5.8 ,   9.6 ,  10.8 ,   0.2 ,
         0.15,   0.1 ,   1.6 ,   8.5 ,   0.25,  22.  ,  19.  ,  23.  ,
         2.8 ,   0.7 ,   1.8 ,   1.2 ,   0.8 ,  10.5 ,  25.  ,  26.  ,
        11.5 ,   8.2 ,   7.3 , 999.  ,  54.5 ,   0.4 ,   7.2 ,   8.6 ,
         3.6 ,   7.5 ,   4.8 ,   6.4 ,   5.2 ,   6.5 ,   9.5 ,   7.4 ,
         4.4 ,  12.5 ,   5.1 ,   6.7 ,   6.6 ,  11.6 ,   6.3 ,  11.4 ,
        13.6 ,   8.01,  14.8 ,  13.5 ,  15.8 ,   9.4 ,   5.01,  10.4 ,
        22.68,  30.  ,  44.  ,   4.2 ,   3.8 ,   5.3 ,   4.9 ,   3.2 ,
         1.9 ,   3.7 ,   4.7 ,  14.5 ,   1.4 ,   3.1 ,   2.7 ,   3.3 ,
         4.6 ,   3.9 ,   2.4 ,   2.2 ,   2.9 ,   2.6 ,   4.3 ,  15.5 ,
      

In [200]:
# 50kg 이상 거래된 건들 이상치라 판단하고 확인 진행
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
3884,오이,55.00000,121000,경남 진주시,certified,2021-05-07,1,2200.00000
93287,포도,999.00000,48000,칠레,normal,2021-05-03,1,48.05000
93589,포도,54.50000,36000,수입산 호주,normal,2021-05-07,1,660.55000
93765,포도,999.00000,48000,칠레,normal,2021-05-12,1,48.05000
93978,포도,999.00000,28000,칠레,normal,2021-05-17,1,28.03000
94279,포도,999.00000,50000,칠레,normal,2021-05-21,1,50.05000
94828,포도,999.00000,46000,칠레,normal,2021-05-29,1,46.05000
295866,바나나,133.00000,23000,수입산 에콰도르,normal,2021-05-03,1,172.93000
441039,오렌지,72.00000,40000,미국,normal,2021-05-04,1,555.56000


In [201]:
df3[(df3['prd']=='오렌지')&(df3['price_kg']<500)]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
440858,오렌지,18.00000,1000,수입산 미국,normal,2021-05-01,1,55.56000
440867,오렌지,17.00000,1000,수입산 미국,normal,2021-05-01,1,58.82000
440958,오렌지,18.00000,1000,미국,normal,2021-05-03,1,55.56000
441107,오렌지,17.00000,1000,수입산 미국,normal,2021-05-05,1,58.82000
441208,오렌지,17.00000,1000,수입산 미국,normal,2021-05-06,1,58.82000
441240,오렌지,18.00000,1000,미국,normal,2021-05-06,1,55.56000
441410,오렌지,17.00000,5000,수입산 미국,normal,2021-05-10,1,294.12000
441458,오렌지,18.00000,1000,미국,normal,2021-05-10,1,55.56000
441806,오렌지,17.00000,1000,수입산 미국,normal,2021-05-20,1,58.82000
441950,오렌지,17.00000,1000,수입산 미국,normal,2021-05-25,1,58.82000


In [202]:
# 스케일 오표기된 항목 수정

# 다른 품목은 확인 결과 문제 없음
# 포도 & 호박만 이상치로 판단하고 모두 삭제
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='호박')].index)
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']=='포도')].index)

In [203]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
3884,오이,55.00000,121000,경남 진주시,certified,2021-05-07,1,2200.00000
295866,바나나,133.00000,23000,수입산 에콰도르,normal,2021-05-03,1,172.93000
441039,오렌지,72.00000,40000,미국,normal,2021-05-04,1,555.56000


### origin

In [204]:
# 도,시, 국산, 수입 표기 분류 필요
df3['origin'].unique()

array(['경기 양주시', '경북 상주시', '충남 천안시', '경기 안성시', '경기 여주시', '경기 연천군',
       '경기 남양주시', '충북 진천군', '충북 청원군', '충남 논산시', '충남 부여군', '충남 공주시',
       '서울 도봉구', '전남 고흥군', '경기 여주군', '경기 이천시', '대전 동구', '대전 서구', '충북 청주시',
       '경기', '충북 음성군', '충남 보령시', '충남 아산시', '경북 문경시', '세종', '경기 평택시',
       '경남 창녕군', '강원 춘천시', '경기 안산시', '충남 서천군', '충남 예산군', '전남 광양시',
       '전남 순천시', '강원 철원군', '전남 구례군', '전북 진안군', '전북 남원시', '전남 나주시',
       '전남 보성군', '서울', '경남 진주시', '경남 고성군', '전남 강진군', '전남 장흥군', '강원 홍천군',
       '경기 양주시 광적면', '충남 천안시 동남구', '경기 파주시', '광주 광산구', '인천 강화군', '충남',
       '경기 양주군', '경기도 안성시', '경기도 양주군', '경기도 평택시', '강원도 철원군', '경기도 고양시',
       '경기도 남양주시', '경기 포천시', '서울 강동구', '경기 의정부', '서울시 도봉구', '충남 천안시 광덕면',
       '경남 합천군', '대전 대덕구', '경기 고양시', '경기도 광명시', '경남 밀양시', '경기 양주시 백석읍',
       '충북 충주시', '충남 부여군 임천면', '전북 임실군', '충남 부여군 세도면', '충남 서산시', '경기 양평군',
       '경기 하남시', '강원 화천군', '인천 강화군 불은면', '경기 광주시', '경기 김포시', '강원 양구군',
       '전북 익산시', '전북 전주시', '경기 안성시 양성면', '전주시 완산구', '강원 횡성군',
       '충남 논산시 가야

In [205]:
# 해당 조건에 맞는 값 추출

splits = df3['origin'].str.split(' ')
cacul = splits.apply(lambda x: pd.Series(x))
df66 = pd.merge(df3,cacul, left_index=True, right_index=True, how='inner')
df66

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,0,1,2,3
0,오이,8.00000,10000,경기 양주시,normal,2021-04-30,1,1250.00000,경기,양주시,NaN,NaN
1,오이,8.00000,11700,경기 양주시,normal,2021-04-30,1,1462.50000,경기,양주시,NaN,NaN
2,오이,18.00000,28000,경북 상주시,normal,2021-04-30,1,1555.56000,경북,상주시,NaN,NaN
3,오이,18.00000,20000,경북 상주시,normal,2021-04-30,1,1111.11000,경북,상주시,NaN,NaN
4,오이,8.00000,13000,충남 천안시,normal,2021-04-30,1,1625.00000,충남,천안시,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
449218,소라,10.00000,65000,국내산,normal,2021-05-27,3,6500.00000,국내산,NaN,NaN,NaN
449219,소라,10.00000,50000,국내산,normal,2021-05-27,3,5000.00000,국내산,NaN,NaN,NaN
449220,소라,10.00000,50000,국내산,normal,2021-05-28,3,5000.00000,국내산,NaN,NaN,NaN
449221,소라,10.00000,107000,전북 군산,normal,2021-05-29,3,10700.00000,전북,군산,NaN,NaN


In [206]:
# 세부 카테고리 정리 (state, city 편집 시 여기서 시작)

df4 = df66.rename(columns = {0:'state',1:'city'})

# 2,3은 필요없는 정보이므로 삭제
df4 = df4.drop(columns=[2,3])

In [207]:
df4['state'].unique()

array(['경기', '경북', '충남', '충북', '서울', '전남', '대전', '세종', '경남', '강원', '전북',
       '광주', '인천', '경기도', '강원도', '서울시', '전주시', '전라북도', '광주시', '대전시',
       '수입산', '필리핀', '베트남', '미국', '대구', '제주', '제주도', '뉴질랜드', '칠레', '페루',
       '우즈베크', '중국', '부산', '부산시', '경상남도', '청주시', '성남시', '인도네시아', '인천광역시',
       '인천시', '호주', '오스트레일', '태국', '전라남도', '제주특별자치도', '세종시', '가락동', '멕시코',
       '우즈베키스탄', '울산', '포항시', '경기고양시일산구', '전북전주시완산구', '충북청주시상당구', '충청남도',
       '타이', '브라질', '대만', '부천시', '콜롬비아', '과테말라', '에콰도르', '코스타리카', '에쿠아도르',
       '에콰돌', '기타외국', '대구시', '경북/문경시', '경북/안동시', '캘리포니아', '수입', '원양산',
       '국내산'], dtype=object)

In [208]:
# 시,도 제거
def name_sort(words):
  p = re.compile('[^시도]+')
  word = re.search(p,str(words))
  return word.group()

df4["state"] = df4["state"].apply(name_sort)

# 오타도 제거

def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["state"] = df4["state"].apply(name_sort2)

In [209]:
df4["state"].unique()

array(['경기', '경북', '충남', '충북', '서울', '전남', '대전', '세종', '경남', '강원', '전북',
       '광주', '인천', '전주', '전라북', '수입산', '필리핀', '베트남', '미국', '대구', '제주',
       '뉴질랜드', '칠레', '페루', '우즈베크', '중국', '부산', '경상남', '청주', '성남', '인',
       '인천광역', '호주', '오스트레일', '태국', '전라남', '제주특별자치', '가락동', '멕', '우즈베키스탄',
       '울산', '포항', '경기고양', '전북전주', '충북청주', '충청남', '타이', '브라질', '대만', '부천',
       '콜롬비아', '과테말라', '에콰', '코스타리카', '에쿠아', '에콰돌', '기타외국', '경북/문경',
       '경북/안동', '캘리포니아', '수입', '원양산', '국내산'], dtype=object)

In [210]:
# state
# 특이케이스 제거

# 국내
df4.loc[df4["state"]=="국내산","city"] = "미상"
df4.loc[df4["state"]=="국내산","state"] = "국산"
df4.loc[df4["state"]=="원양산","city"] = "미상"

## 도단위
df4.loc[df4["state"]=="충청북","state"] = "충북"
df4.loc[df4["state"]=="충청남","state"] = "충남"
df4.loc[df4["state"]=="전라북","state"] = "전북"
df4.loc[df4["state"]=="전라남","state"] = "전남"
df4.loc[df4["state"]=="경상북","state"] = "경북"
df4.loc[df4["state"]=="경상남","state"] = "경남"

## 시단위
### 광역시

df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="대구","city"] = "대구"
df4.loc[df4["state"]=="서울특별","city"] = "서울"
df4.loc[df4["state"]=="서울특별","state"] = "서울"
df4.loc[df4["state"]=="인천광역","city"] = "인천"
df4.loc[df4["state"]=="인천광역","state"] = "인천"
df4.loc[df4["state"]=="부산","city"] = "부산"
df4.loc[df4["state"]=="가락동","city"] = "서울"
df4.loc[df4["state"]=="가락동","state"] = "서울"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="제주","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","state"] = "제주"
df4.loc[df4["state"]=="제주특별자","city"] = "제주"
df4.loc[df4["state"]=="제주특별자","state"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","city"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","state"] = "제주"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="울산","city"] = "울산"
df4.loc[df4["state"]=="울산","state"] = "울산"

### 일반시
df4.loc[df4["state"]=="성남","city"] = "성남"
df4.loc[df4["state"]=="성남","state"] = "경기"
df4.loc[df4["state"]=="전주","city"] = "전주"
df4.loc[df4["state"]=="전주","state"] = "전북"
df4.loc[df4["state"]=="청주","city"] = "청주"
df4.loc[df4["state"]=="청주","state"] = "충북"
df4.loc[df4["state"]=="수원","city"] = "수원"
df4.loc[df4["state"]=="수원","state"] = "경기"
df4.loc[df4["state"]=="부천","city"] = "부천"
df4.loc[df4["state"]=="부천","state"] = "경기"
df4.loc[df4["state"]=="경기고양","city"] = "고양"
df4.loc[df4["state"]=="경기고양","state"] = "경기"
df4.loc[df4["state"]=="충북청주","city"] = "청주"
df4.loc[df4["state"]=="충북청주","state"] = "충북"
df4.loc[df4["state"]=="전북전주","city"] = "전주"
df4.loc[df4["state"]=="전북전주","state"] = "전북"
df4.loc[df4["state"]=="경북/문경","city"] = "문경"
df4.loc[df4["state"]=="경북/문경","state"] = "경북"
df4.loc[df4["state"]=="포항","city"] = "포항"
df4.loc[df4["state"]=="포항","state"] = "경북"
df4.loc[df4["state"]=="경북/안동","city"] = "안동"
df4.loc[df4["state"]=="경북/안동","state"] = "경북"


# 외국
## 아시아
df4.loc[df4["state"]=="칠레","city"] = "칠레"
df4.loc[df4["state"]=="칠레","state"] = "수입"
df4.loc[df4["state"]=="일본","city"] = "일본"
df4.loc[df4["state"]=="일본","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="우즈베크","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베크","state"] = "수입"
df4.loc[df4["state"]=="이스라엘","city"] = "이스라엘"
df4.loc[df4["state"]=="이스라엘","state"] = "수입"
df4.loc[df4["state"]=="터키","city"] = "터키"
df4.loc[df4["state"]=="터키","state"] = "수입"
df4.loc[df4["state"]=="중국","city"] = "중국"
df4.loc[df4["state"]=="중국","state"] = "수입"
df4.loc[df4["state"]=="타이","city"] = "태국"
df4.loc[df4["state"]=="타이","state"] = "수입"
df4.loc[df4["state"]=="태국","city"] = "태국"
df4.loc[df4["state"]=="태국","state"] = "수입"
df4.loc[df4["state"]=="필리핀","city"] = "필리핀"
df4.loc[df4["state"]=="필리핀","state"] = "수입"
df4.loc[df4["state"]=="우즈베키스탄","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베키스탄","state"] = "수입"
df4.loc[df4["state"]=="대만","city"] = "대만"
df4.loc[df4["state"]=="대만","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="이란","city"] = "이란"
df4.loc[df4["state"]=="이란","state"] = "수입"
df4.loc[df4["state"]=="러","city"] = "러시아"
df4.loc[df4["state"]=="러","state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"city"] = "인도네시아"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"city"] = "인도"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"state"] = "수입"

## 아메리카

df4.loc[df4["state"]=="캘리포니아","city"] = "미국"
df4.loc[df4["state"]=="캘리포니아","state"] = "수입"
df4.loc[df4["state"]=="포클랜드","city"] = "영국"
df4.loc[df4["state"]=="포클랜드","state"] = "수입"
df4.loc[df4["state"]=="콜롬비아","city"] = "콜롬비아"
df4.loc[df4["state"]=="콜롬비아","state"] = "수입"
df4.loc[df4["state"]=="에콰","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰","state"] = "수입"
df4.loc[df4["state"]=="에쿠아","city"] = "에콰도르"
df4.loc[df4["state"]=="에쿠아","state"] = "수입"
df4.loc[df4["state"]=="에콰돌","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰돌","state"] = "수입"
df4.loc[df4["state"]=="과테말라","city"] = "과테말라"
df4.loc[df4["state"]=="과테말라","state"] = "수입"
df4.loc[df4["state"]=="코스타리카","city"] = "코스타리카"
df4.loc[df4["state"]=="코스타리카","state"] = "수입"
df4.loc[df4["state"]=="멕","city"] = "멕시코"
df4.loc[df4["state"]=="멕","state"] = "수입"
df4.loc[df4["state"]=="페루","city"] = "페루"
df4.loc[df4["state"]=="페루","state"] = "수입"
df4.loc[df4["state"]=="미국","city"] = "미국"
df4.loc[df4["state"]=="미국","state"] = "수입"
df4.loc[df4["state"]=="아르헨티나","city"] = "아르헨티나"
df4.loc[df4["state"]=="아르헨티나","state"] = "수입"
df4.loc[df4["state"]=="브라질","city"] = "브라질"
df4.loc[df4["state"]=="브라질","state"] = "수입"

## 유럽
df4.loc[df4["state"]=="스페인","city"] = "스페인"
df4.loc[df4["state"]=="스페인","state"] = "수입"
df4.loc[df4["state"]=="우크라이나","city"] = "우크라이나"
df4.loc[df4["state"]=="우크라이나","state"] = "수입"
df4.loc[df4["state"]=="이탈리아","city"] = "이탈리아"
df4.loc[df4["state"]=="이탈리아","state"] = "수입"
df4.loc[df4["state"]=="불가리아","city"] = "불가리아"
df4.loc[df4["state"]=="불가리아","state"] = "수입"

## 아프리카
df4.loc[df4["state"]=="남아프리카","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카","state"] = "수입"
df4.loc[df4["state"]=="남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카공화국","state"] = "수입"
df4.loc[df4["state"]=="수입남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="수입남아프리카공화국","state"] = "수입"


## 오세아니아
df4.loc[df4["state"]=="뉴질랜드","city"] = "뉴질랜드"
df4.loc[df4["state"]=="뉴질랜드","state"] = "수입"
df4.loc[df4["state"]=="오스트레일","city"] = "호주"
df4.loc[df4["state"]=="오스트레일","state"] = "수입"
df4.loc[df4["state"]=="오스트레일리아","city"] = "호주"
df4.loc[df4["state"]=="오스트레일리아","state"] = "수입"
df4.loc[df4["state"]=="호주","city"] = "호주"
df4.loc[df4["state"]=="호주","state"] = "수입"
df4.loc[df4["state"]=="통가","city"] = "통가"
df4.loc[df4["state"]=="통가","state"] = "수입"
df4.loc[df4["state"]=="뉴칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴칼레","state"] = "수입"
df4.loc[df4["state"]=="뉴","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴","state"] = "수입"
df4.loc[df4["state"]=="누벨칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="누벨칼레","state"] = "수입"
# 기타
df4.loc[df4["state"]=="수입산","state"] = "수입"
df4.loc[df4["state"]=="기타외국","city"] = "미상"
df4.loc[df4["state"]=="기타외국","state"] = "수입"

In [211]:
df4[df4['state']=='nan']

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city


In [212]:
df4['state'].unique()

array(['경기', '경북', '충남', '충북', '서울', '전남', '대전', '세종', '경남', '강원', '전북',
       '광주', '인천', '수입', '대구', '제주', '부산', '울산', '원양산', '국산'],
      dtype=object)

In [213]:
# city
# 우선 결측치 제거
df4.loc[df4["city"].isnull(),"city"] = "미상"

In [214]:
# 이후 시,군 등 제거
def name_sort(words):
  p = re.compile('[^시군]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort)

In [215]:
# \t 도 제거
def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort2)

In [216]:
df4['city'].unique()

array(['양주', '상주', '천안', '안성', '여주', '연천', '남양주', '진천', '청원', '논산', '부여',
       '공주', '도봉구', '고흥', '이천', '대전', '청주', '미상', '음성', '보령', '아산', '문경',
       '세종', '평택', '창녕', '춘천', '안산', '서천', '예산', '광양', '순천', '철원', '구례',
       '진안', '남원', '나주', '보성', '진주', '고성', '강진', '장흥', '홍천', '파주', '광산구',
       '강화', '고양', '포천', '강동구', '의정부', '합천', '광명', '밀양', '충주', '임실', '서산',
       '양평', '하남', '화천', '광주', '김포', '양구', '익산', '전주', '횡성', '위', '당진',
       '여수', '순창', '남해', '양양', '순', '진도', '태안', '홍성', '남구', '베트남', '필리핀',
       '미국', '송파구', '영천', '영동', '김천', '경산', '대구', '함안', '함양', '제주', '의령',
       '무안', '제천', '하동', '평창', '구미', '인제', '고창', '강서구', '뉴질랜드', '성주',
       '칠곡', '고령', '안동', '해남', '신안', '영월', '용인', '칠레', '페루', '경주',
       '우즈베키스탄', '영암', '화성', '부안', '청양', '완주', '완도', '태백', '구리', '계양구',
       '양천구', '부천', '괴산', '성남', '산청', '금산', '북구', '중국', '부산', '담양', '영광',
       '옥천', '산', '화순', '사천', '김해', '정읍', '장수', '김제', '함평', '서구', '가평',
       '마산', '창원', '장성', '청도', '오산', '영등포구', '무주', '중구'

In [217]:
# 나머지 케이스 정리

# 국내
df4.loc[df4["city"]=="기타","city"] = "미상"
## 일반시
df4.loc[df4["city"]=="위","city"] = "군위"
df4.loc[df4["city"]=="고흥(나로도)","city"] = "고흥"
df4.loc[df4["city"]=="산","city"] = "군산"
df4.loc[df4["city"]=="장항(서천)","city"] = "서천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="순천(광양)","city"] = "순천"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="화성(평택)","city"] = "화성"
df4.loc[df4["city"]=="흥","city"] = "시흥"
df4.loc[df4["city"]=="목포(무안)","city"] = "목포"
df4.loc[df4["city"]=="목포(무안)","state"] = "전남"
df4.loc[df4["city"]=="보성(벌교)","city"] = "보성"
df4.loc[df4["city"]=="울진(후포)","city"] = "울진"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="순","city"] = "순창"
df4.loc[df4["city"]=="미금","city"] = "성남"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="남"),"city"] = "남원"
df4.loc[(df4['prd']=="호박") & (df4['state']=='전북'),"city"] = "전주"
## 광역시
df4.loc[df4["city"]=="강서구","city"] = "서울"
df4.loc[df4["city"]=="은평구","city"] = "서울"
df4.loc[df4["city"]=="강동구","city"] = "서울"
df4.loc[df4["city"]=="마포구","city"] = "서울"
df4.loc[df4["city"]=="중구","city"] = "서울"
df4.loc[df4["city"]=="송파구","city"] = "서울"
df4.loc[df4["city"]=="도봉구","city"] = "서울"
df4.loc[df4["city"]=="중랑구","city"] = "서울"
df4.loc[df4["city"]=="양천구","city"] = "서울"
df4.loc[df4["city"]=="성동구","city"] = "서울"
df4.loc[df4["city"]=="강남구","city"] = "서울"
df4.loc[df4["city"]=="동대문구","city"] = "서울"
df4.loc[df4["city"]=="영등포구","city"] = "서울"
df4.loc[df4["city"]=="영등포","city"] = "서울"
df4.loc[df4["city"]=="서초구","city"] = "서울"
df4.loc[df4["city"]=="구로구","city"] = "서울"
df4.loc[df4["city"]=="동대문","city"] = "서울"
df4.loc[df4["city"]=="금천구","city"] = "서울"
df4.loc[df4["city"]=="동작구","city"] = "서울"

df4.loc[df4["city"]=="광산구","city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="남구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="북구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="서구"),"city"] = "광주"
df4.loc[df4["city"]=="동구","city"] = "광주"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="남구"),"city"] = "인천"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="서구"),"city"] = "인천"
df4.loc[df4["city"]=="미추홀","city"] = "인천"
df4.loc[df4["city"]=="부평구","city"] = "인천"
df4.loc[df4["city"]=="남동구","city"] = "인천"
df4.loc[df4["city"]=="미추홀구","city"] = "인천"
df4.loc[df4["city"]=="계양구","city"] = "인천"
df4.loc[df4["city"]=="마산(고성,진해)","city"] = "창원"
df4.loc[(df4["state"]=="부산")&(df4["city"]=="북구"),"city"] = "부산"
df4.loc[df4["city"]=="오포","city"] = "부산"
df4.loc[df4["city"]=="부산진","city"] = "부산"
df4.loc[df4["city"]=="사상구","city"] = "부산"
df4.loc[df4["city"]=="부산진구","city"] = "부산"
df4.loc[df4["city"]=="북제주","city"] = "제주"
df4.loc[df4["city"]=="남제주","city"] = "제주"

# 해외
## 아시아
df4.loc[df4["city"]=="러","city"] = "러시아"
df4.loc[df4["city"]=="말레이지아","city"] = "말레이시아"
df4.loc[df4["city"]=="타이","city"] = "태국"
df4.loc[df4["city"]=="인도네","city"] = "인도네시아"
## 아메리카
df4.loc[df4["city"]=="멕","city"] = "멕시코"
## 아프리카
df4.loc[df4["city"]=="남아공산","city"] = "남아공"
## 유럽
## 오세아니아

In [218]:
# 기존 city 리스트와 비교
ex_city = pd.read_csv("../cities.csv")
df4[~df4['city'].isin(ex_city['city'])]['city'].unique()

array(['우즈베키스탄', '브라질', '말레이시아'], dtype=object)

In [219]:
# 리스트 최신화
new_city = pd.DataFrame(df4['city'].unique(),columns=['city'])
cities = ex_city.append(new_city,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_8897/1856950528.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = ex_city.append(new_city,ignore_index=True)


In [220]:
cities.drop(columns=['Unnamed: 0'],inplace=True)
cities.drop_duplicates(inplace=True)
cities.reset_index(inplace=True,drop=True)

In [221]:
cities.to_csv('../cities.csv')

In [222]:
# 처리 완료하였으니 df4는 백업으로 두고 df5 활용
df5 = df4.drop(columns='origin')

# 전처리 결과

# 파일 저장

In [223]:
df5.to_csv('../data_proceed/202105_proceed.csv')